In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_gen = ImageDataGenerator(rescale =(1/255.))
train_dir = "./training_set/"
test_dir = "./test_set/"

In [2]:
train_gen = image_gen.flow_from_directory(directory = train_dir,
                                          batch_size=32,
                                          target_size = (224,224),
                                          classes=['cats','dogs'],
                                          class_mode='binary',
                                          seed = 2022)
val_gen = image_gen.flow_from_directory(test_dir,
                                         batch_size=32,
                                         target_size = (224,224),
                                         classes=['cats','dogs'],
                                         class_mode='binary',
                                         seed = 2022)


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [5]:
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.convolutions1 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32,(3,3),padding='same', input_shape=(224,224,3)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2)),
            
        ])
        self.convolutions2 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64,(3,3),padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2))
        ])
        self.convolutions3 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64,(3,3),padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2))
        ])
        
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(256),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation= 'sigmoid')
        ])
    def call(self,x):
        x = self.convolutions1(x)
        x = self.convolutions2(x)
        x = self.convolutions3(x)
        x = self.classifier(x)
        return x
    
        

In [6]:
model = CNN()
model.build(input_shape=(128,224,224,3))

In [7]:
model.summary()

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 112, 112, 32)      1024      
_________________________________________________________________
sequential_1 (Sequential)    multiple                  18752     
_________________________________________________________________
sequential_2 (Sequential)    multiple                  37184     
_________________________________________________________________
sequential_3 (Sequential)    multiple                  12845569  
Total params: 12,902,529
Trainable params: 12,902,209
Non-trainable params: 320
_________________________________________________________________


In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
def schedule(epoch, lr):
    if epoch <5:
        return lr 
    elif epoch < 10:
        return lr*tf.math.exp(-0.1)
    else:
        return lr*tf.math.exp(-0.01)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(schedule)
earlystopping = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='tmp_checkpoint.ckpt',
                                                save_weights_only=True,
                                                save_best_only=True,
                                                monitor = 'val_loss',
                                                verbose = 1)

In [9]:
model.compile(optimizer=optimizer, loss = tf.losses.BinaryCrossentropy(from_logits=True), metrics = ['accuracy'])

In [12]:
history = model.fit(train_gen, callbacks=[lr_schedule, earlystopping, checkpoint],validation_data=val_gen, epochs = 20)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 251 steps, validate for 64 steps
Epoch 1/20
 33/251 [==>...........................] - ETA: 5:04 - loss: 0.6931 - accuracy: 0.4707WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr


KeyboardInterrupt: 

---
### 전이 학습 모델

In [14]:
pre_trained_base = tf.keras.applications.ResNet50V2(include_top=False,
                                                    weights = 'imagenet',
                                                    input_shape =[224,224,3])
pre_trained_base.trainable = False

In [15]:
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.top = tf.keras.Sequential([
            pre_trained_base,
        ])    
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(64),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1, activation = 'sigmoid')
        ])
    def call(self,x):
        x = self.top(x)
        x = self.classifier(x)
        return x

In [16]:
model = CNN()
model.build(input_shape=(224,224,3))

In [17]:
model.summary()

Model: "cnn_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_4 (Sequential)    (None, 7, 7, 2048)        23564800  
_________________________________________________________________
sequential_5 (Sequential)    multiple                  12853505  
Total params: 36,418,305
Trainable params: 12,853,505
Non-trainable params: 23,564,800
_________________________________________________________________


In [18]:
model.compile(optimizer=optimizer, loss = tf.losses.BinaryCrossentropy(from_logits=True), metrics = ['accuracy'])

In [19]:
history = model.fit(train_gen, callbacks=[lr_schedule, earlystopping, checkpoint],validation_data=val_gen, epochs = 20)

  ...
    to  
  ['...']
Train for 251 steps, validate for 64 steps
Epoch 1/20
250/251 [============================>.] - ETA: 3s - loss: 0.6933 - accuracy: 0.4992
Epoch 00001: val_loss did not improve from 0.69315
251/251 [==============================] - 876s 3s/step - loss: 0.6933 - accuracy: 0.4996 - val_loss: 0.6931 - val_accuracy: 0.4998
Epoch 2/20
250/251 [============================>.] - ETA: 3s - loss: 0.6931 - accuracy: 0.4993
Epoch 00002: val_loss did not improve from 0.69315
251/251 [==============================] - 874s 3s/step - loss: 0.6931 - accuracy: 0.4997 - val_loss: 0.6931 - val_accuracy: 0.4998
Epoch 3/20
250/251 [============================>.] - ETA: 3s - loss: 0.6931 - accuracy: 0.4993
Epoch 00003: val_loss did not improve from 0.69315
251/251 [==============================] - 911s 4s/step - loss: 0.6931 - accuracy: 0.4997 - val_loss: 0.6931 - val_accuracy: 0.4998
Epoch 4/20
250/251 [============================>.] - ETA: 3s - loss: 0.6931 - accuracy: 0.5002

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_acc(history):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot training & validation accuracy values
    ax[0].plot(history.history['accuracy'])
    ax[0].plot(history.history['val_accuracy'])
    ax[0].set_title('Model accuracy')
    ax[0].set_ylabel('Accuracy')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Val'], loc='upper left')
    
    # Plot training & validation loss values
    ax[1].plot(history.history['loss'])
    ax[1].plot(history.history['val_loss'])
    ax[1].set_title('Model loss')
    ax[1].set_ylabel('Loss')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Val'], loc='upper left')
    
    plt.show()
plot_loss_acc(history) 